In [ ]:
import gzip
import numpy as np
from glob import glob
import pandas as pd
from io import StringIO
from os.path import basename
import matplotlib.pyplot as plt
from scipy import interpolate
from os.path import join as fullfile
from tqdm import tqdm
import os
import cv2

In [ ]:
img_dir = glob('/hdd1/Projects/yqiu_intern/March2021_Appen/TrustVideos/TrustSemanticSegementation/*v3/images', recursive = False)

In [ ]:
# Read jsom file of objID

  
import json
  
# Opening JSON file
f = open('objID_2_rgb_map.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
  
# Closing file
f.close()
inv_data = {v: int(k) for k, v in data.items()}
inv_data['[255, 0, 0]'] = 30
inv_data['[0, 255, 0]'] = 220 # 90 
inv_data['[0, 0, 255]'] = 230
inv_data['[0, 0, 0]'] = 210 # 180 

In [ ]:
def add_mask(seg_img):
#     seg_img = cv2.imread(seg_img_path + Synced_df['ImageFile'].values[i])
        # draw a polygon for Dashboard
    vertices = np.array([[0, 310], [222, 545], [636, 468], [900,468], [1280,505], [1280,720], [0, 720]], np.int32)
        # vertices = np.array([[480, 400], [250, 650], [600, 650]], np.int32)
    pts = vertices.reshape((-1, 1, 2))
    cv2.polylines(seg_img, [pts], isClosed=False, color=(188, 210, 253), thickness=0)
        # fill it
    cv2.fillPoly(seg_img, [pts], color=(188, 210, 253))

        # draw a circle for Speed # color (0, 0, 0)
    circle_speed_w, circle_speed_h = 243, 210
    circle_speed_radius = 50
    cv2.circle(seg_img,(circle_speed_w,circle_speed_h),circle_speed_radius,(0, 0, 0),thickness=-1)

        # draw a circle for Arrow # color (0, 255, 0)
    circle_arrow_w, circle_arrow_h = 775, 552
    circle_arrow_radius = 30
    cv2.circle(seg_img,(circle_arrow_w,circle_arrow_h),circle_arrow_radius,(0, 0, 255),thickness=-1)

        # draw a circle for Speedometer
    circle_speedoMeter_w, circle_speedoMeter_h = 410, 648
    circle_speedoMeter_radius = 70
    cv2.circle(seg_img,(circle_speedoMeter_w,circle_speedoMeter_h),circle_speedoMeter_radius,(0, 0, 0),thickness=-1)
        #save new segmentation image
#     cv2.imwrite('segmentaion_img_with_mask/'+Synced_df['ImageFile'].values[i], seg_img)
        
#     Added_image = cv2.addWeighted(seg_img,0.5,rgb_img,1,0)
    return seg_img

def create_seg_img(seg_img, scale_percent):
    width = int(seg_img.shape[1] * scale_percent / 100)
    height = int(seg_img.shape[0] * scale_percent / 100)
    dim = (width, height)

    resized_seg_img = cv2.resize(seg_img, dim, interpolation = cv2.INTER_NEAREST)
    gray_img = np.zeros((height, width))

    for i in range(len(resized_seg_img)): # 270
        for j in range(len(resized_seg_img[0])): # 480
            inv_data_key = str(list(resized_seg_img[i, j, :]))
            gray_img[i][j] = int(inv_data[inv_data_key])/255
    
    return gray_img # 1 channel image

In [ ]:
for folder in img_dir:
    print(folder)
    imgs = glob(folder + '/**', recursive = False)
    for img in imgs:
#         print(img[-31:])
        seg_img = cv2.imread(img)
        seg_img_msk = add_mask(seg_img)
        seg_img = create_seg_img(seg_img_msk, 100)
        cv2.imwrite('/home/yqiu/Yuning_HRI_Intern/data/seg_img_720_1280_new_dash/'+img[-31:], seg_img*255)

        

